In [2]:
from transformers import AutoTokenizer, AutoModel
import torch
import re
import torch.nn.functional as F

def load_model(model_path):
    """
    加载模型和分词器。

    参数:
    model_path (str): 模型的路径。

    返回:
    tokenizer: 加载的分词器。
    model: 加载的模型。
    """
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModel.from_pretrained(model_path)
    model.eval()
    return tokenizer, model

def get_embeddings(prompts, tokenizer, model):
    """
    计算给定提示的嵌入表示。

    参数:
    prompts (list): 自定义提示的列表。
    tokenizer: 加载的分词器。
    model: 加载的模型。

    返回:
    torch.Tensor: 嵌入表示的张量。
    """
    # Tokenize custom prompts
    encoded_input = tokenizer(prompts, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        model_output = model(**encoded_input)
        # Perform pooling. In this case, cls pooling.
        sentence_embeddings = model_output[0][:, 0]

    # Normalize embeddings
    sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

    return sentence_embeddings

def calculate_cosine_similarity(embedding_1, embeddings):
    """
    计算一个嵌入与其他多个嵌入的余弦相似度。

    参数:
    embedding_1 (torch.Tensor): 第一个嵌入表示。
    embeddings (torch.Tensor): 其他嵌入表示的张量。

    返回:
    torch.Tensor: 相似度列表。
    """
    cosine_similarities = F.cosine_similarity(embedding_1.unsqueeze(0), embeddings)
    return cosine_similarities

In [3]:
import pandas as pd
def get_theme_prompts(lda_key_words_path):
    file_path = lda_key_words_path
    df = pd.read_excel(file_path)
    
    prompts = []
    
    for index, row in df.iterrows():
        theme_category = row['主题类别']
        keywords = row.dropna()[1:].tolist()
        prompt = f"新闻的主题是{theme_category}，关键词是{'、'.join(keywords)}"
        prompts.append(prompt)
    

    # 返回6个prompt
    return prompts


In [17]:
# 分隔句子
def split_sentences(text):
    # 使用正则表达式分隔句子
    sentences = re.split(r'[！。？；]', text)
    # 去掉空字符串
    sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
    return sentences

# 处理新闻文本并计算相似度
def process_news_data(news_file_path, lda_key_words_path, model_path, output_file_path):
    # 加载模型和分词器
    tokenizer, model = load_model(model_path)
    
    # 读取新闻文本的Excel文件
    df = pd.read_excel(news_file_path,nrows = 10)
    
    # 获取主题提示
    key_topic_list = get_theme_prompts(lda_key_words_path)
    
    # 检查主题提示列表是否为空
    if not key_topic_list:
        raise ValueError("主题提示列表为空，请检查Excel文件内容是否正确。")
    
    # 获取主题提示的嵌入
    key_topic_embeddings = get_embeddings(key_topic_list, tokenizer, model)
    
    # 遍历每一行
    for index, row in df.iterrows():
        # 分隔句子
        news_content = row['NewsContent']
        sentences = split_sentences(news_content)
        # 存储分隔后的句子
        df.at[index, 'NewsContent_cut'] = str({f"news_prompt{i+1}": sentence for i, sentence in enumerate(sentences)})
        
        # 存储每个句子的主题匹配结果
        topic_match_dict = {}
        for sentence in sentences:
            # 获取句子的嵌入
            sentence_embedding = get_embeddings([sentence], tokenizer, model)
            
            # 计算相似度
            similarity_scores = calculate_cosine_similarity(sentence_embedding, key_topic_embeddings)
            print(similarity_scores)
            # 检查相似度分数是否为空
            if similarity_scores.numel() == 0:
                raise ValueError("相似度分数为空，请检查模型输出是否正确。")
            
            # 找出相似度最高的主题
            max_similarity_index = torch.argmax(similarity_scores).item()
            most_similar_topic = key_topic_list[max_similarity_index]
            
            # 存储结果
            topic_match_dict[sentence] = most_similar_topic
        
        # 存储到新的列
        df.at[index, 'cut_news_topic_match'] = str(topic_match_dict)
    
    # 保存结果到新的Excel文件
    df.to_excel(output_file_path, index=False)
    print("处理完成，结果已保存到", output_file_path)

In [6]:
import torch
from modelscope import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('/root/.cache/LLMS/hub/BAAI/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('/root/.cache/LLMS/hub/BAAI/bge-reranker-v2-m3')
model.eval()

pairs = [['what is panda?', 'panda is an animal'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

tensor([4.9582, 5.2650])


In [12]:
import faiss
print(f"FAISS version: {faiss.__version__}")
print(f"FAISS GPU count: {faiss.get_num_gpus()}")

FAISS version: 1.8.0
FAISS GPU count: 0


In [1]:
import torch
from modelscope import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('/root/.cache/LLMS/hub/BAAI/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('/root/.cache/LLMS/hub/BAAI/bge-reranker-v2-m3')
model.eval()

pairs = [['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']]
with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    print(scores)

/root/miniconda3/envs/news_emo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


tensor([-8.1838,  5.2650])


In [18]:
if __name__ == "__main__":
    # 模型路径
    model_path = '/root/.cache/LLMS/hub/BAAI/bge-large-zh-v1___5'  # BGE-large-zh-v1.5模型
    
    # 新闻文本文件路径
    news_file_path = '/root/LLM_news_emo_analyze/DATA/news_test_data.xlsx'
    
    # 主题提示文件路径
    lda_key_words_path = '~/LLM_news_emo_analyze/DATA/lda_key_words.xlsx'
    
    # 输出文件路径
    output_file_path = '~/LLM_news_emo_analyze/DATA/news_data_with_topics.xlsx'
    
    # 调用主处理函数
    process_news_data(news_file_path, lda_key_words_path, model_path, output_file_path)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['A股首例市值退市股即将摘牌', '9月1日晚间，*ST深天（000023，股价：1.72元', '总市值2.387亿元）公告称，公司股票已被深圳证券交易所决定终止上市，并将于2024年9月2日摘牌', '公告显示，6月27日至7月24日，*ST深天通过深交所交易系统连续二十个交易日的股票收盘市值均低于3亿元，触及深交所《股票上市规则(2023年8月修订)》第9.2.1条第一款第六项规定的股票终止上市情形', '根据深交所《股票上市规则(2024年修订)》第9.2.7条的规定以及深交所上市审核委员会的审议意见，深交所决定终止公司股票上市', '同时，根据相关规定，公司股票因触及交易类强制退市情形被作出终止上市决定，不进入退市整理期', '值得注意的是，*ST深天此番退市，为A股公司市值退市首例，创造A股市场新的历史', '根据目前仍在执行的规定，公司连续二十个交易日在深交所的股票收盘市值均低于3亿元，深交所终止其股票上市交易', '而根据深交所此前发布的《深圳证券交易所股票上市规则（2024年修订）》，如果A股公司“连续二十个交易日在本所的股票收盘总市值均低于5亿元”，就会被终止上市，关于市值退市情形的修订将自2024年10月30日起计算相关期限，意味着未来上市公司触及市值退市的概率会更高', '*ST深天是一家以商品混凝土为主业、房地产为支柱产业的上市公司，主要产业包括商品混凝土的生产和销售、房地产的开发及物业管理等', '2020年起，*ST深天业绩就持续呈现亏损局面', '2024年上半年，该公司亏损额已经超越营收规模，实现营业收入4083.54万元，同比下滑54.35%', '净利润亏损7032.84万元', '据*ST深天披露，2024年上半年，受地产行业需求下滑影响，深圳市和株洲市混凝土行业内生产能力的过剩更加严重，导致市场竞争更加激烈', '公司混凝土业务受市场需求不足、公司资金压力加大等不利因素的影响，企业运行艰难程度不断增加', '在房地产板块，*ST深天1984年起涉足房地产业，与其他公司合作开发了一批商住楼宇', '而后公司设立专门从事房地产开发与经营的工程开发公司，自成立以来，先后开发了数个住宅、工业、商业物业', '公司现有房地产业务主要以商用物业及普通住宅为主，业务区域主要集中在深圳、西安和连云港', '值得注意的是，8月30日，*ST

IndexError: list index out of range

In [18]:
if __name__ == "__main__":
    lda_key_words_path = '~/LLM_news_emo_analyze/DATA/lda_key_words.xlsx'
    news_prompt = "证券时报网讯，中信建投研报指出，2024年上半年，根据样本数据，医疗器械板块收入同比增长1%，扣非归母净利润同比增长3%。"
    key_topic_lisy = get_theme_prompts(lda_key_words_path)
    model_path = '/root/.cache/LLMS/hub/BAAI/bge-large-zh-v1___5'  # BGE-large-zh-v1.5模型
    tokenizer, model = load_model(model_path)

    # 获取所有提示的嵌入
    all_prompts = [news_prompt] + key_topic_lisy
    embeddings = get_embeddings(all_prompts, tokenizer, model)

    # 计算prompt_1与其他提示的相似度
    similarity_scores = calculate_cosine_similarity(embeddings[0], embeddings[1:])

    # 找出相似度最高的提示及其相似度值
    max_similarity_index = torch.argmax(similarity_scores).item()
    max_similarity_score = similarity_scores[max_similarity_index].item()
    most_similar_prompt = key_topic_lisy[max_similarity_index]

    print("与news_prompt最相似的提示是:", most_similar_prompt)
    print("相似度值:", max_similarity_score)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


与news_prompt最相似的提示是: 新闻的主题是生产投资，关键词是制造、设备、研发、制造业、工业、生产线、机械、材料、设计、国外、钢铁、产能
相似度值: 0.39552390575408936


In [21]:
DATA = pd.read_excel('/root/LLM_news_emo_analyze/DATA/news_data_with_topics.xlsx')
DATA[['NewsContent_cut','cut_news_topic_match']]

,NewsContent_cut,cut_news_topic_match
0,"{'news_prompt1': 'A股首例市值退市股即将摘牌', 'news_prompt...",{'A股首例市值退市股即将摘牌': {'topic': '新闻的主题是金融，关键词是A股、股...
1,{'news_prompt1': '又见A股上市公司实控人被留置，涉事公司分别是半导体设备供...,{'又见A股上市公司实控人被留置，涉事公司分别是半导体设备供应商耐科装备（688419.SH...
2,{'news_prompt1': '摘要 \u3000\u3000国常会：研究推动保险业高质...,{'摘要 \u3000\u3000国常会：研究推动保险业高质量发展的若干意见': {'top...
3,{'news_prompt1': '据目前安排，若无变化本周（9月2日到6日）共有3只新股申...,{'据目前安排，若无变化本周（9月2日到6日）共有3只新股申购': {'topic': '新...
4,{'news_prompt1': '证券时报网讯，中信建投研报指出，2024年上半年，根据样...,{'证券时报网讯，中信建投研报指出，2024年上半年，根据样本数据，医疗器械板块收入同比增长...
5,{'news_prompt1': '证券时报网讯，证券时报·数据宝统计显示，社保基金最新出现...,{'证券时报网讯，证券时报·数据宝统计显示，社保基金最新出现在605只个股前十大流通股东名单...
6,{'news_prompt1': '证券时报网讯，中信证券研报指出，2024年以来，家电内需...,{'证券时报网讯，中信证券研报指出，2024年以来，家电内需渐入“地产竣工周期负反馈”': ...
7,{'news_prompt1': '证券时报网讯，中信建投研报指出，经历数月回调后，9月市场...,{'证券时报网讯，中信建投研报指出，经历数月回调后，9月市场有望边际改善': {'topic...
8,{'news_prompt1': '证券时报网讯，证券时报·数据宝统计显示，社保基金最新出现...,{'证券时报网讯，证券时报·数据宝统计显示，社保基金最新出现在66只科创板股前十大流通股东名...
9,"{'news_prompt1': '日前，美国证券交易委员会（SEC）披露了一则指控', '...",{'日前，美国证券交易委员会（SEC）披露了一则指控': {'topic': '新闻的主题是...


In [ ]:
lda_key_words_path = '~/LLM_news_emo_analyze/DATA/lda_key_words.xlsx'
def find_most_similar_prompt(input_prompt,lda_key_words_path):
    theme_prompts = get_theme_prompts(lda_key_words_path)
    max_similarity = -1
    most_similar_prompt = None
    for theme_prompt in theme_prompts:
        embeddings = get_embeddings([input_prompt, theme_prompt], tokenizer, model)
        similarity = calculate_cosine_similarity(embeddings)
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_prompt = theme_prompt

return most_similar_prompt, max_similarity

In [ ]:
    for theme_prompt in theme_prompts:
        embeddings = get_embeddings([input_prompt, theme_prompt], tokenizer, model)
        similarity = calculate_cosine_similarity(embeddings)
        
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_prompt = theme_prompt
    
    return most_similar_prompt, max_similarity

In [9]:
lda_key_words_path = '~/LLM_news_emo_analyze/DATA/lda_key_words.xlsx'


In [ ]:
if __name__ == "__main__":
    prompt_1 = "这是第一个自定义的提示"
    prompt_2 = "这是第二个自定义的提示"
    model_path = '/root/.cache/LLMS/hub/BAAI/bge-large-zh-v1___5' # BG-large-zh-v1.5模型
    
    tokenizer, model = load_model(model_path)
    embeddings = get_embeddings([prompt_1, prompt_2], tokenizer, model)
    
    similarity = calculate_cosine_similarity(embeddings)
    print("余弦相似度:", similarity)